# Flux off project

The pupose of this jupyter notebook is to provide a proof of concept to map the circuit breaker trip time to the magnetic field of the device.

In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks
%matplotlib qt5



# Directory to scan
directory = "samples/19_5/"
folder='plots'
if not os.path.exists(folder):
        os.makedirs(folder)
# List to store file paths and names
file_paths = []
diff_times = []
# Scan the directory
for root, dirs, files in os.walk(directory):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)



selected_file_paths = file_paths

# Print the file paths and names
for file_path in selected_file_paths[0:1]:
    print("File path:", file_path)
    print("File name:", os.path.basename(file_path))
    df = pd.read_csv(file_path)

    sample_times_us = df.iloc[:, 0].values #convert the data into 1D numpy array
    sample_times_us = sample_times_us - sample_times_us[0]
    data_raw = df.iloc[:, 1].values #convert the data into 1D numpy array

    #data_ma = data_ma - np.mean(data_ma)
    time_taken_seconds = (sample_times_us[-1] - sample_times_us[0]) / 1e6
    dt = np.mean(np.diff(sample_times_us)) / 1e6
    sample_times_s = sample_times_us / 1e6

    data = data_raw

    print("Time taken from first to last sample:", time_taken_seconds, "seconds")
    print("dt:", dt, "seconds")
    print("Sample Freqency:", 1/dt, "Hz")

  

    # Perform FFT
    fft_result1 = np.fft.fft((data - np.mean(data))/np.max(data))
    n = data.size
    fft_freq = np.fft.fftfreq(n, dt)

    # Extract the positive side of the FFT result
    fft_result1 = fft_result1[:len(fft_result1)//2]
    fft_freq = fft_freq[:len(fft_freq)//2]

    plt.figure(1,figsize=(15, 10))
    plt.semilogy(fft_freq, np.abs(fft_result1))
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (log scale)')
    plt.xlim(0, (1/dt)/ 2)  # Show frequencies up to Nyquist frequency
    plt.ylim(10**-1,np.abs(np.max(fft_result1)))
    #Set frequency ticks to 50 Hz intervals
    min_freq = np.floor(fft_freq.min() / 50) * 50
    max_freq = np.ceil(fft_freq.max() / 50) * 50
    plt.xticks(np.arange(min_freq, max_freq + 1, 50))
    plt.grid()
    plt.tight_layout()

    indices_5s = np.where(sample_times_s < 5)[0]

    time_5s = sample_times_s[indices_5s]
    data_5s = data[indices_5s]

    z = [0,546]

    y_pred = z[0] * time_5s + z[1]

    diff = data_5s - y_pred


    fig, axs = plt.subplots(2, 1, figsize=(10, 8))  # 2x2 grid of subplots

    axs[0].hist(np.abs(diff), bins=50, edgecolor='black')
    axs[0].set_title('Distrobution of differnece')
    axs[0].grid()

    # Second subplot
    axs[1].plot(time_5s,data_5s)
    axs[1].plot(time_5s,y_pred)
    axs[1].set_title('Histogram of Random Data 2')    
    axs[1].grid()

    plt.tight_layout()
    plt.show()


    peaks_data, _ = find_peaks(data_5s,height = 540,distance=4)


    time_5s = time_5s[peaks_data]
    data_5s = data_5s[peaks_data]
    y_pred = z[0] * time_5s + z[1]
    diff = data_5s - y_pred


    fig, axs = plt.subplots(2, 1, figsize=(10, 8))  # 2x2 grid of subplots

    axs[0].hist(np.abs(diff), bins=5, edgecolor='black')
    axs[0].set_title('Distrobution of differnece')
    axs[0].grid()

    # Second subplot
    axs[1].plot(time_5s,data_5s)
    axs[1].plot(time_5s,y_pred)
    axs[1].set_title('Histogram of Random Data 2')    
    axs[1].grid()

    plt.tight_layout()
    plt.show()



    

File path: samples/19_5/19_5A_0.csv
File name: 19_5A_0.csv
Time taken from first to last sample: 74.3845 seconds
dt: 0.0005 seconds
Sample Freqency: 2000.0 Hz
